In [15]:
import torch
import torchvision
import torchvision.models as models
from PIL import Image
import matplotlib.pyplot as plt
import utils
import numpy as np
import os

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
transform = torchvision.transforms.Compose(
    [
        torchvision.transforms.ToTensor()
    ]
)

## Resnet

### Edit Model for Last Layer Hook

In [42]:
resnet18 = models.resnet18(weights=models.ResNet18_Weights.DEFAULT).to(device)

In [43]:
resnet18

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [44]:
resnet18.layer4[1]

BasicBlock(
  (conv1): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [45]:
last_activations = []
def get_last_layer_activation(module, input, output):
    last_activations.append(output)

In [46]:
resnet18.layer4[1].register_forward_hook(get_last_layer_activation)

### Run Inference

In [47]:
pkl12path = 'mappings/exhaustive_mapping_12.pkl'
pkl12 = utils.load_dict(pkl12path)

In [50]:
last_activations = []

In [ ]:
folpath = 'exhaustive_generated_set\\12'
for transformation in utils.HAHN_TRANSFORMATION_COMBOS:
    print(f'-------------Running {transformation}-------------------')
    if not 'randominsertion' in transformation:
        for filepath in os.listdir(folpath):
            imgpath = os.path.join(folpath, filepath)
            imgin = Image.open(imgpath)
            imgin = transform(imgin).unsqueeze(0).to(device)

            imglist = pkl12[filepath.split('.')[0]]
            print(imglist)
            for t in transformation:
                func = getattr(utils, t)
                imglist = func(imglist)
            print(imglist)
            imgout = utils.array2img(imglist, num_circles=len(imglist), radius=0.75)
            imgout = transform(imgout).unsqueeze(0).to(device)

            with torch.no_grad():
                resnet18(imgin)
                resnet18(imgout)
            assert len(last_activations) == 2
            tensin = last_activations[0]
            tensout = last_activations[1]
            last_activations = []

1


AssertionError: 

In [33]:
img = Image.open("exhaustive_generated_set\\00a02a4c.png")

In [34]:
img = transform(img).unsqueeze(0).to(device)

In [35]:
with torch.no_grad():
    resnet18(img)

In [40]:
last_activations[0].shape

torch.Size([1, 512, 7, 7])

## VGG

In [ ]:
vgg16 = models.vgg16(pretrained=True)

## DenseNet

In [ ]:
densenet121 = models.densenet121(pretrained=True)

## Inception

In [ ]:
inception_v3 = models.inception_v3(pretrained=True)

## AlexNet

In [ ]:
alexnet = models.alexnet(pretrained=True)